In [ ]:
import torch
import os
import shutil
from Loader import TrainLoader
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
from Network import Net
import torch.nn as nn
import cv2
import torchvision.utils as utils
import torchvision

In [ ]:
# dataset at https://www.kaggle.com/datasets/navoneel/brain-mri-images-for-brain-tumor-detection
main_dir = "D:\\Workshops\\CNN_workshop\\workshop"

In [ ]:
def data_split(main_dir, train_ratio, val_ratio, test_ration):
    yes_dir = os.path.join(main_dir, "archive", "yes")
    no_dir = os.path.join(main_dir, "archive", "no")
    yes_names = os.listdir(yes_dir)
    no_names = os.listdir(no_dir)
    len_yes = len(yes_names)
    len_no = len(no_names)
    
    yes_train_names = yes_names[:int(train_ratio*len_yes)]
    no_train_names = no_names[:int(train_ratio*len_no)]
    
    yes_val_names = yes_names[int(train_ratio*len_yes):int((train_ratio+val_ratio)*len_yes)]
    no_val_names = no_names[int(train_ratio*len_no):int((train_ratio+val_ratio)*len_no)]
    
    yes_test_names = yes_names[int((train_ratio+val_ratio)*len_yes):]
    no_test_names = no_names[int((train_ratio+val_ratio)*len_no):]
    
    # creating data folder
    data_dir = os.path.join(main_dir, "data")
    train_dir = os.path.join(data_dir, 'train')
    val_dir = os.path.join(data_dir, 'val')
    test_dir = os.path.join(data_dir, 'test')
    if not os.path.exists(data_dir):
        os.mkdir(data_dir)
        os.mkdir(train_dir)
        os.mkdir(val_dir)
        os.mkdir(test_dir)
        
    # copying train data
    for yes_train_sample in yes_train_names:
        source = os.path.join(yes_dir, yes_train_sample)
        destination = os.path.join(train_dir, yes_train_sample)
        shutil.copy(source, destination)
    for no_train_sample in no_train_names:
        source = os.path.join(no_dir, no_train_sample)
        destination = os.path.join(train_dir, no_train_sample)
        shutil.copy(source, destination)  
        
    for yes_val_sample in yes_val_names:
        source = os.path.join(yes_dir, yes_val_sample)
        destination = os.path.join(val_dir, yes_val_sample)
        shutil.copy(source, destination)
    for no_val_sample in no_val_names:
        source = os.path.join(no_dir, no_val_sample)
        destination = os.path.join(val_dir, no_val_sample)
        shutil.copy(source, destination) 
        
    for yes_test_sample in yes_test_names:
        source = os.path.join(yes_dir, yes_test_sample)
        destination = os.path.join(test_dir, yes_test_sample)
        shutil.copy(source, destination)
    for no_test_sample in no_test_names:
        source = os.path.join(no_dir, no_test_sample)
        destination = os.path.join(test_dir, no_test_sample)
        shutil.copy(source, destination)  
    
    
    return yes_names, no_names

In [ ]:
y, n = data_split(main_dir, 0.6, 0.2, 0.2)

In [ ]:
batch_size = 32

In [ ]:
train_path = "D:\\Workshops\\CNN_workshop\\workshop\\data\\train"
val_path = "D:\\Workshops\\CNN_workshop\\workshop\\data\\val"
test_path = "D:\\Workshops\\CNN_workshop\\workshop\\data\\test"
train_loader_obj = TrainLoader(train_path, y, n)
train_loader = DataLoader(dataset=train_loader_obj, batch_size = batch_size, shuffle=True)
val_loader_obj = TrainLoader(val_path, y, n)
val_loader = DataLoader(val_loader_obj, batch_size = 32, shuffle = True)
test_loader_obj = TrainLoader(test_path, y, n)
test_loader = DataLoader(test_loader_obj, batch_size = 32, shuffle = True)


In [ ]:
iterator = iter(val_loader)
images, labels = next(iterator)

In [ ]:
images, labels = next(iterator)

In [ ]:
plt.imshow(images[0])

# Creating Network

In [ ]:
net = Net()
device = "cuda"
net = net.to(device)

In [ ]:
pytorch_total_params = sum(p.numel() for p in net.parameters() if p.requires_grad)
print(pytorch_total_params)

In [ ]:
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.01, momentum=0.9)

# Train the neural network
for epoch in range(500):
    for i, data in enumerate(train_loader, 0):
        # Get the inputs and labels
        inputs, labels = data
        labels = labels.to(device)
        inputs = inputs.to(device).float()
        inputs = inputs.permute(0, 3, 1, 2)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Print statistics


print('Finished Training')

In [ ]:
#weight_name = "Conv_weights.pt"
#torch.save(net.state_dict(), weight_name)
net.load_state_dict(torch.load("Conv_weights.pt"))

In [ ]:
net

In [ ]:
net.eval()
iterator = iter(test_loader)
images, labels = next(iterator)

inputs = images.to(device).float()
inputs = inputs.permute(0, 3, 1, 2)
with torch.no_grad():
    outputs = net(inputs)
    _, predicted = torch.max(outputs.data, 1)
    
print(labels)
print(predicted)

In [ ]:
diff = predicted.to(device) - labels.to(device)
acc = 1- sum(abs(diff))/len(predicted)
print(acc)

In [ ]:
input_image = cv2.imread("D:\\Workshops\\CNN_workshop\\workshop\\data\\test\\Y82.jpg")
plt.imshow(input_image)
input_image = input_image/255.0
input_image = (input_image - np.mean(input_image))/np.std(input_image)
input_image = torch.from_numpy(input_image)
input_image = input_image.unsqueeze(0).float().to(device)
input_image = input_image.permute(0, 3, 1, 2)
conv3_output = net.conv3(net.pool(nn.functional.relu(net.conv2(net.pool(nn.functional.relu(net.conv1(input_image))))))).detach()
conv3_output = conv3_output.squeeze()
conv3_output = conv3_output.detach().to('cpu')
conv3_output = np.array(conv3_output)

In [ ]:
conv1_output = net.conv1(input_image).detach()
conv1_output = conv1_output.squeeze()
conv1_output = conv1_output.detach().to('cpu')
conv1_output = np.array(conv1_output)

In [ ]:
plt.imshow(conv1[12], vmin=0, vmax=1)
plt.show()

In [ ]:
plt.imshow(conv1[5], vmin=0, vmax=1)
plt.show()

# Transfer Learning

In [ ]:
from ReNet_model import ResNet

In [ ]:
net = ResNet()
device = "cuda"
net = net.to(device)

In [ ]:
pytorch_total_params = sum(p.numel() for p in net.parameters() if p.requires_grad)
print(pytorch_total_params)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

# Train the neural network
epoch_acc = 0
iterator = iter(val_loader)
images_val, labels_val = next(iterator)
for epoch in range(500):
    for i, data in enumerate(train_loader, 0):
        # Get the inputs and labels
        inputs, labels = data
        labels = labels.to(device)
        inputs = inputs.to(device).float()
        inputs = inputs.permute(0, 3, 1, 2)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        
    net.eval()

    inputs = images_val.to(device).float()
    inputs = inputs.permute(0, 3, 1, 2)
    with torch.no_grad():
        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, 1) 
    
    diff = predicted.to(device) - labels_val.to(device)
    acc = 1- sum(abs(diff))/len(predicted)
    print("Accuracy of epoch {} is {}".format(epoch+1, acc*100))
    
    if acc > epoch_acc:
        weight_name = "best_model.pt"
        torch.save(net.state_dict(), weight_name)
        epoch_acc = acc
    
print('Finished Training')

In [ ]:
net.load_state_dict(torch.load("best_model.pt"))

In [ ]:
net.eval()
iterator = iter(test_loader)
images, labels = next(iterator)

inputs = images.to(device).float()
inputs = inputs.permute(0, 3, 1, 2)
with torch.no_grad():
    outputs = net(inputs)
    _, predicted = torch.max(outputs.data, 1)
    
print(labels)
print(predicted)

In [ ]:
diff = predicted.to(device) - labels.to(device)
acc = 1- sum(abs(diff))/len(predicted)
print(acc)

In [ ]:
input_image = cv2.imread("D:\\Workshops\\CNN_workshop\\workshop\\data\\test\\Y82.jpg")
input_image = input_image/255.0
input_image = (input_image - np.mean(input_image))/np.std(input_image)
input_image = torch.from_numpy(input_image)
input_image = input_image.unsqueeze(0).float().to(device)
input_image = input_image.permute(0, 3, 1, 2)

In [ ]:
net.resnet[:4]

In [ ]:
conv1_output = net.resnet[:4][0](input_image).detach()
conv1_output = conv1_output.squeeze()
conv1_output = conv1_output.detach().to('cpu')
conv1_output = np.array(conv1_output)

In [ ]:
plt.imshow(conv1_output[1], vmin=0, vmax=1)
plt.show()

In [ ]:
plt.imshow(conv1_output[19], vmin=0, vmax=1)
plt.show()

In [ ]:
plt.imshow(conv1_output[24], vmin=0, vmax=1)
plt.show()

In [ ]:
plt.imshow(conv1_output[28], vmin=0, vmax=1)
plt.show()